In [1]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import numpy as np
import pandas as pd
import re
import os
import random
from collections import Counter

import tensorflow as tf
from tensorflow import gfile

In [17]:
location = "hugo" # "urban" "amsterdam" "jungle" 
# if we make a new location / or project we need to do the following:
# create a csv labels file
# create a csv labels prop file
# create the new directories that hold the new train and eval samples
#
# for example the new project `hugo`
# in ..serval/dataset/classlabels/csv_files we need 
# class_labels_indices_hugo.csv
# class_labels_indices_hugo_prop.csv (copy of class_labels_indices_hugo.csv)
# in this csv we need to add a column `target_ind` and put all zeros, this is a placeholder of the propertions per class
# train_hugo_new and eval_hugo_new in serval/dataset


In [18]:
# inside docker goto right place 
os.chdir("/tf/serval/serval")
# set relvant full paths
# /tf is the tensorflow jupyter container mapping
path_original_tf_records = "dataset/audioset_v1_embeddings/"
path_new_tf_records = "dataset/"
path_class_labels = "dataset/classlabels/csv_files/"
#os.getcwd()

In [19]:
## Paden naar originele tf-records: bal_train, unbal_train en eval
bal_train_data_pattern = path_original_tf_records + "bal_train/*.tfrecord"
unbal_train_data_pattern = path_original_tf_records + "unbal_train/*.tfrecord"
eval_data_pattern = path_original_tf_records + "eval/*.tfrecord"

## Paden voor nieuwe tf-records (train en test)
train_pattern_new = path_new_tf_records + "train_" + location + "_new"
eval_pattern_new = path_new_tf_records + "eval_" + location + "_new"

In [20]:
## Boolean die aangeeft of alle examples worden geschreven naar een tf-record
## Bij false: upsamplen/downsamplen adhv proporties
write_all_examples = True
## proportie train/test
pct_train = 0.6

In [21]:
## Lijst van alle labels (nieuwe) en alle combi's
all_labels = []
label_combinations = []

In [22]:
## Open csv-bestanden met class labels
## Oorspronkelijke csv-file google audioset class labels
mid_to_label_old = pd.read_csv(path_class_labels + "class_labels_indices_original_audioset.csv", sep=",")

## csv-file met labels (urban/jungle/amsterdam)
new_label_file = "class_labels_indices_" + location + ".csv"
mid_to_label_new = pd.read_csv(path_class_labels + new_label_file, sep=";")
## mid_to_label_new = pd.read_csv('csv_files/class_labels_indices_urban.csv', sep=";")

## Naam bestand om proporties naartoe te schrijven (of om te gebruiken bij sampelen)
## Als deze nog niet bestaat: maak een kopietje van mid_to_labels_new met de naam die hieronder staat
name_label_prop_file = "class_labels_indices_" + location + "_prop.csv"
label_proportions = pd.read_csv(path_class_labels + name_label_prop_file, sep=";")

In [23]:
mid_to_label_new

,index,mid,display_name
0,0,/t/dd00134,"Car, boat, motorcyle, truck"
1,1,/m/01j4z9,Chainsaw
2,2,/m/07pczhz,Chop
3,3,/m/032s66,"Gunshot, gunfire"
4,4,/m/09ct_,Helicopter
5,5,/t/dd00003,Male singing
6,6,/m/07rpkh9,Cattle
7,0,/m/02rlv9,"Car, boat, motorcyle, truck"
8,0,/m/04_sv,"Car, boat, motorcyle, truck"
9,7,/m/06bz3,Radio


In [10]:
## Deze functie checkt of een example wordt gemaakt (downsamplen)
def checkIfNewExample(labels):
    
    ## Alleen bij 1 label worden er examples overgeslagen (of bij write_all_examples)
    if(len(labels)>1 or write_all_examples):
        return True
    else:
        label = list(labels)[0]
        prop = label_proportions.loc[label_proportions['index']==np.int(label), "target_ind"].values[0]
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:
            return True
        else:
            return False
    return False

In [11]:
## Deze functie checkt of er nog een example moet worden gemaakt (upsamplen)
## Momenteel kan het aantal enkel verdubbeld worden

def checkIfExtraExample(labels):
    
    ## Alleen bij 1 label worden er examples gekopieerd (of als er niet (up)gesampled wordt)
    if(len(labels)>1 or write_all_examples):
        return False
    else:
        label = list(labels)[0]
        prop = label_proportions.loc[label_proportions['index']==np.int(label), "target_ind"].values[0]-1
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:    
            return True
        else:
            return False
    return False

In [12]:
#### Find new label for each of old label (527 -> x)

def getNewLabel(label_old):
    
    ## Get m-id that matches with index
    mid = mid_to_label_old.loc[mid_to_label_old['index']==label_old, 'mid']

    ## Find the new label belonging to this m-id
    label_new = mid_to_label_new.loc[mid_to_label_new['mid']==str(mid.values[0]), 'index']
    
    return label_new

In [13]:
def createExample(example_old, new_labels):
    
    ## Bouw structuur van tf-record met video-id, labels en features
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})
    #print("New labels: " + str(new_labels))
    
    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [14]:
## Deze code haalt voor een label alle (grand)parents op
## Momenteel worden deze functies niet meer gebruikt.
## Maar schroom je niet om eens te kijken hoe deze prachtige recursieve functie werkt!

def getParents(label, parents):
    
    parent = int(mid_to_label_new.loc[mid_to_label_new['index']==label, 'parent'])
    
    if parent==999:
        return parents
    else:
        parents.append(parent)
        return getParents(parent, parents)
    return parent

def getLabelsWithParents(labels):
    total_labels = labels
    for label in labels:
        parents = getParents(label, [])
        total_labels.extend(parents)
    return(set(total_labels))

In [32]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## 1 tf-record heeft meerdere examples
## Deze functie checkt voor elk example of het een label heeft uit onze labelset

## Input: filename van tf-record (pad) en prefix (vanwege dubbele filenames)
## Result: schrijven van nieuw tf-record

## Geef prefix mee (om dubbele namen te voorkomen)
def readWriteTfRecord(tfrecords_filename, prefix):
    
    ## Maak 2 writers (train en test) die nieuw tf-records schrijft
    
    # windows 
    #record_name = tfrecords_filename.split('\\')[-1]
    # linux
    record_name = tfrecords_filename.split('/')[-1]
    ## Let op: hij doet moeilijk over AUX.tfrecord. Dus prefix A bij UX mag niet
    #if(str(prefix+record_name_=="AUX.tfrecord"):
    #    record_name="UXx.tfrecord"
        
    ## Nieuwe filenames: 1 voor train en 1 voor eval
    train_file_new = str(train_pattern_new + '/' + prefix + record_name)
    eval_file_new = str(eval_pattern_new + '/' + prefix + record_name)

    train_writer = tf.python_io.TFRecordWriter(train_file_new)
    eval_writer = tf.python_io.TFRecordWriter(eval_file_new)
    nr_new_examples = 0
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
        
        new_labels = []
        for label in labels:
            ## Hij returnt een set van cellen (of leeg of van 1 rij)
            label_new = getNewLabel(label)
            
            if(label_new.shape[0]>0):
                new_labels.append(label_new.values[0])
        
        ## Als enkele (grand)parents ontbreken, voeg ze toe
        #new_labels = getLabelsWithParents(new_labels)
        ## If any of the labels left (for our model), write new tf-record
        
        if (len(new_labels) > 0 and checkIfNewExample(new_labels)):
        #if(len(new_labels) > 0):
            ## Add to all labels
            all_labels.extend(new_labels)
            label_combinations.append(new_labels)
   
            sequence_example = createExample(example, new_labels)
            nr_new_examples += 1
            
            ## Write the example
            rand = random.random()
            if rand <= pct_train:
                train_writer.write(sequence_example.SerializeToString())
            else:
                eval_writer.write(sequence_example.SerializeToString())
        
            ## Check of er nog een example moet worden gemaakt (upsamplen)
            if(checkIfExtraExample(new_labels)):
                rand = random.random()
                if rand <= pct_train:
                    train_writer.write(sequence_example.SerializeToString())
                else:
                    eval_writer.write(sequence_example.SerializeToString())
        
                all_labels.extend(new_labels)
                label_combinations.append(new_labels)
            print('Example gemaakt! Video-id: ' + str(example.context.feature['video_id'].bytes_list.value)
                  + ' labels : ' + str(new_labels))

    #print("nr new examples: " + str(nr_new_examples))
    
    # Only write if any examples created

    if nr_new_examples > 0:
        train_writer.close()
        eval_writer.close()
        #print("tfrecord written")
    else:
        #print("empty file.. remove: " + path_tfrecord_new)
        os.remove(train_file_new)
        os.remove(eval_file_new)

In [33]:
## Maak nieuwe tf-records van bal_train
# make sure the new directory location exists train_amsterdam_new
# ISSUE amsterdam has new labels? csv not ; separated

files = gfile.Glob(bal_train_data_pattern)

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'B')

Example gemaakt! Video-id: [b'OMo7qvmsMu4'] labels : [10, 0]
Example gemaakt! Video-id: [b'cEHnqMIacho'] labels : [10, 0]
Example gemaakt! Video-id: [b'L3Xr_NVi7UM'] labels : [10, 0]
Example gemaakt! Video-id: [b'Xy8fR4pJNmo'] labels : [10, 0]
Example gemaakt! Video-id: [b'-99daJhXYJY'] labels : [10, 0]
Example gemaakt! Video-id: [b'SFeGH1FEJVk'] labels : [0, 1]
Example gemaakt! Video-id: [b'a6HE939n-Jk'] labels : [10, 0]
Example gemaakt! Video-id: [b'4tQiC9E6Mxc'] labels : [10, 0]
Example gemaakt! Video-id: [b'OplcFe9OOMA'] labels : [10, 0]
Example gemaakt! Video-id: [b'H0adZlMICzE'] labels : [10, 0]
Example gemaakt! Video-id: [b'ISUgb6HjLbI'] labels : [10, 0]
Example gemaakt! Video-id: [b'UCuylpOa2IE'] labels : [10, 0]
Example gemaakt! Video-id: [b'c65F3Bvn2y8'] labels : [10, 0]
Example gemaakt! Video-id: [b'mIkW7mWlnXw'] labels : [10, 3]
Example gemaakt! Video-id: [b'MtXS_LO3nNU'] labels : [10, 0]
Example gemaakt! Video-id: [b'sWN5-jMQCgs'] labels : [10, 4]


In [36]:
## Maak nieuwe tf-records van unbal_train
# LETOP deze is groot en duurt lang, voor tests niet nodig

files = gfile.Glob(unbal_train_data_pattern)

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'U')

Example gemaakt! Video-id: [b'7bwmQoybmtU'] labels : [10, 0]
Example gemaakt! Video-id: [b'ObBLNd2cTrM'] labels : [10, 0]
Example gemaakt! Video-id: [b'IRXtZg_AdeQ'] labels : [10, 0]
Example gemaakt! Video-id: [b'SZCaGRPUT-o'] labels : [10, 0]
Example gemaakt! Video-id: [b'ycrwL0gpWe4'] labels : [10, 0]
Example gemaakt! Video-id: [b'19qNZuPUkcY'] labels : [10, 0]
Example gemaakt! Video-id: [b'7SFpXsyLKAw'] labels : [10, 0]
Example gemaakt! Video-id: [b'BYt1uSiqnmw'] labels : [10, 0]
Example gemaakt! Video-id: [b'Sc5ERPGmbsk'] labels : [10, 0]
Example gemaakt! Video-id: [b'MLMesd8-7Ls'] labels : [10, 4]
Example gemaakt! Video-id: [b'MLCDc310iMI'] labels : [10, 0]
Example gemaakt! Video-id: [b'MhiIKBzmIpI'] labels : [10, 0]
Example gemaakt! Video-id: [b'RuUaCEmGYRI'] labels : [10, 0]
Example gemaakt! Video-id: [b'YQkawB5CvpU'] labels : [10, 0]
Example gemaakt! Video-id: [b'YQqw0qBoyZs'] labels : [10, 0]
Example gemaakt! Video-id: [b'RI01_heS550'] labels : [10, 0]
Example gemaakt! Video-i

Example gemaakt! Video-id: [b'yO4X40RZ0FE'] labels : [10, 0]
Example gemaakt! Video-id: [b'x8BKOJ4gU4E'] labels : [10, 0]
Example gemaakt! Video-id: [b'Fz0W9C1PUBY'] labels : [10, 0]
Example gemaakt! Video-id: [b'QCAIEfIjRYg'] labels : [10, 0]
Example gemaakt! Video-id: [b'0WB8KgBvZDk'] labels : [10, 0]
Example gemaakt! Video-id: [b'4_YFwkoYfPI'] labels : [10, 0]
Example gemaakt! Video-id: [b'L9TuQouWKFE'] labels : [10, 0]
Example gemaakt! Video-id: [b'Hoq1UoeU4Ds'] labels : [10, 0]
Example gemaakt! Video-id: [b'TfESpuZhSTE'] labels : [0, 9]
Example gemaakt! Video-id: [b'CsjWzXhJ4Ow'] labels : [10, 0]
Example gemaakt! Video-id: [b'CsZEJL88_Qg'] labels : [4, 3]
Example gemaakt! Video-id: [b'Css-VjwUi8w'] labels : [10, 0]
Example gemaakt! Video-id: [b'9tH_ZjUW_yo'] labels : [10, 0]
Example gemaakt! Video-id: [b'QX_Eu-s7-u8'] labels : [10, 0]
Example gemaakt! Video-id: [b'EgzDUT5Fh3E'] labels : [10, 0]
Example gemaakt! Video-id: [b'ErYz-dQRdQU'] labels : [10, 0]
Example gemaakt! Video-id:

Example gemaakt! Video-id: [b'Atmv7HYs-uk'] labels : [10, 0]
Example gemaakt! Video-id: [b'P0Nvk6Dk6Bw'] labels : [10, 0]
Example gemaakt! Video-id: [b'mcIc5-e2_UI'] labels : [10, 0]
Example gemaakt! Video-id: [b'iF7TZSU1qPo'] labels : [10, 0]
Example gemaakt! Video-id: [b'HaKezfKkwRk'] labels : [10, 0]
Example gemaakt! Video-id: [b'iCDWoKau-8g'] labels : [9, 7]
Example gemaakt! Video-id: [b'OZyyY6KdZ9k'] labels : [10, 0]
Example gemaakt! Video-id: [b'B5HskNKCj8c'] labels : [0, 0]
Example gemaakt! Video-id: [b'obh88Lqof0E'] labels : [10, 0]
Example gemaakt! Video-id: [b'M5Bw_a_6PTQ'] labels : [0, 4]
Example gemaakt! Video-id: [b'BoOi4aG_Lt8'] labels : [10, 0]
Example gemaakt! Video-id: [b'zHhpM_siNfk'] labels : [10, 0]
Example gemaakt! Video-id: [b'YHge6P8lzaI'] labels : [10, 0]
Example gemaakt! Video-id: [b'YH0OsR_K-Eo'] labels : [10, 0]
Example gemaakt! Video-id: [b'Q78Vy9-8Ykw'] labels : [10, 0]
Example gemaakt! Video-id: [b'zXheGQDY7AU'] labels : [10, 0]
Example gemaakt! Video-id: 

Example gemaakt! Video-id: [b'tqQs_C92FLQ'] labels : [10, 0]
Example gemaakt! Video-id: [b'8YUH8PhpQ7g'] labels : [10, 0]
Example gemaakt! Video-id: [b'JCQ5jK4kI9w'] labels : [10, 0]
Example gemaakt! Video-id: [b'CUT6_UsanZA'] labels : [0, 0]
Example gemaakt! Video-id: [b'CBBiSzV8YEM'] labels : [0, 1]
Example gemaakt! Video-id: [b'PebA2Cu07cI'] labels : [10, 0]
Example gemaakt! Video-id: [b'OAfi-5sHdS8'] labels : [0, 4]
Example gemaakt! Video-id: [b'0J3iRi1U0vw'] labels : [10, 0]
Example gemaakt! Video-id: [b'VAHEmWtO-6w'] labels : [10, 0]
Example gemaakt! Video-id: [b'N3OzszGvsx0'] labels : [10, 0]
Example gemaakt! Video-id: [b'QxqBjW1dA0w'] labels : [10, 0]
Example gemaakt! Video-id: [b'Qxgf65htq94'] labels : [10, 0]
Example gemaakt! Video-id: [b'C-cTm9ruSFM'] labels : [10, 0]
Example gemaakt! Video-id: [b'0irDRK0J2NI'] labels : [10, 0]
Example gemaakt! Video-id: [b'fm0vcJHKkvw'] labels : [10, 0]
Example gemaakt! Video-id: [b'IKYAMHcgzKo'] labels : [0, 1]
Example gemaakt! Video-id: [

Example gemaakt! Video-id: [b'J3U_vm80oXE'] labels : [10, 0]
Example gemaakt! Video-id: [b'sdk62fv8Upw'] labels : [10, 0]
Example gemaakt! Video-id: [b'FbkRnLYap8o'] labels : [4, 9]
Example gemaakt! Video-id: [b'HEJPKEKRq6w'] labels : [10, 0]
Example gemaakt! Video-id: [b'3NPidVvhTnI'] labels : [10, 0]
Example gemaakt! Video-id: [b'mXsx1Wj-n8k'] labels : [10, 0]
Example gemaakt! Video-id: [b'KVCEkvej6jA'] labels : [10, 0]
Example gemaakt! Video-id: [b'nD0bEGbA3LQ'] labels : [0, 0]
Example gemaakt! Video-id: [b'2Pm60iUCmAw'] labels : [10, 0]
Example gemaakt! Video-id: [b'Nm5r_0KzwKU'] labels : [10, 0]
Example gemaakt! Video-id: [b'SMRrRSx3uF8'] labels : [10, 0]
Example gemaakt! Video-id: [b'nhOYEQSa5YY'] labels : [10, 0]
Example gemaakt! Video-id: [b'D9Gr4pntwk4'] labels : [0, 0]
Example gemaakt! Video-id: [b'D9zJiy_8OWk'] labels : [10, 0]
Example gemaakt! Video-id: [b'gal9p6n7jXY'] labels : [10, 0]
Example gemaakt! Video-id: [b'-z639l6WR2w'] labels : [0, 1]
Example gemaakt! Video-id: [

Example gemaakt! Video-id: [b'qGD4usQxmTQ'] labels : [0, 9]
Example gemaakt! Video-id: [b'9V9129KCcSw'] labels : [10, 0]
Example gemaakt! Video-id: [b'K_IJIHcx-Fc'] labels : [10, 0]
Example gemaakt! Video-id: [b'3rEm-LU0YsU'] labels : [10, 0]
Example gemaakt! Video-id: [b'n7CbU6TM7Lw'] labels : [6, 0]
Example gemaakt! Video-id: [b'G9obMIjfP9Q'] labels : [0, 9]
Example gemaakt! Video-id: [b'G9p5a5366SA'] labels : [10, 7]
Example gemaakt! Video-id: [b'FyUtHq2GnZI'] labels : [10, 0]
Example gemaakt! Video-id: [b'IZXG2Bs2nu0'] labels : [10, 0]
Example gemaakt! Video-id: [b'2hsNSgL5NaI'] labels : [0, 1]
Example gemaakt! Video-id: [b'2hlmelcd7BE'] labels : [10, 0]
Example gemaakt! Video-id: [b'4Db5ZWCfND4'] labels : [10, 0]
Example gemaakt! Video-id: [b'7glcEETAOwA'] labels : [10, 0]
Example gemaakt! Video-id: [b'K65qJK66mEI'] labels : [10, 0]
Example gemaakt! Video-id: [b'6EREUtlV0DQ'] labels : [0, 4]
Example gemaakt! Video-id: [b'COPFHHhsIK4'] labels : [10, 0]
Example gemaakt! Video-id: [b

In [34]:
## Maak nieuwe tf-records van eval

files = gfile.Glob(eval_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'E')

Example gemaakt! Video-id: [b'ycqDMKTrvLY'] labels : [0, 7]
Example gemaakt! Video-id: [b'SRjsxeOWFnw'] labels : [10, 0]
Example gemaakt! Video-id: [b'AS0PySOANPo'] labels : [10, 0]
Example gemaakt! Video-id: [b'owZDBEq6WdU'] labels : [0, 1]
Example gemaakt! Video-id: [b'AkUDv7JexjQ'] labels : [10, 0]
Example gemaakt! Video-id: [b'd_Hdkt1_ivo'] labels : [10, 0]
Example gemaakt! Video-id: [b'BTNz6NftP34'] labels : [0, 4]
Example gemaakt! Video-id: [b'3t6pqBKV0kY'] labels : [10, 0]
Example gemaakt! Video-id: [b'Hfa-bfdZtKI'] labels : [10, 0]
Example gemaakt! Video-id: [b'8hAk1TZk8pM'] labels : [10, 0]
Example gemaakt! Video-id: [b'9F1Tw0VnG38'] labels : [10, 0]
Example gemaakt! Video-id: [b'-Z1ZSWDouUU'] labels : [10, 0]
Example gemaakt! Video-id: [b'h5NTDVXiTiI'] labels : [4, 1]
Example gemaakt! Video-id: [b'4BO6EbCTKwM'] labels : [10, 0]
Example gemaakt! Video-id: [b'9L21uB0nyq0'] labels : [10, 0]
Example gemaakt! Video-id: [b'q3qm4bSfpu8'] labels : [10, 0]
Example gemaakt! Video-id: [

In [38]:
## Print voor alle labels het aantal voorkomens in de nieuwe tf-records

sum_occur = 0
cnt_labels = Counter(all_labels)

for i in cnt_labels.most_common(100):
    sum_occur = sum_occur + i[1]
    print(label_proportions.loc[label_proportions['index']==i[0]]['display_name'].values[0], ' : ', str(i[1]))

Car, boat, motorcyle, truck  :  1147
Wind noise (microphone)  :  947
Chainsaw  :  50
Silence  :  49
Helicopter  :  35
Gunshot, gunfire  :  11
Radio  :  11
Male singing  :  5
Cattle  :  4
Shout  :  3


In [87]:
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [88]:
## Houd individuele voorkomens bij (dus aantal keer dat label voorkomt zonder andere labels)
ind_occur = [0 for x in range(label_proportions.shape[0])] 

## Geef meest voorkomende combinaties
cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(20):
    label_names = []
    
    ## Maak van string weer een lijst (zie vorige cel)
    label_list = label_combination[0].split("_")
    
    ## Code die alleen individuele labels telt
    if(len(label_list)==1):
        ind_occur[int(label_list[0])] = label_combination[1]
    
    for i in label_list:
        label_names.append(str(label_proportions.loc[label_proportions['index']==i]['display_name']))
    label_names = ', '.join(label_names)
    print(label_names,': ', label_combination[1])

Series([], Name: display_name, dtype: object) :  2525
Series([], Name: display_name, dtype: object) :  2508
Series([], Name: display_name, dtype: object) :  2486
Series([], Name: display_name, dtype: object) :  2477
Series([], Name: display_name, dtype: object) :  2475
Series([], Name: display_name, dtype: object) :  2465
Series([], Name: display_name, dtype: object) :  2463
Series([], Name: display_name, dtype: object) :  2458
Series([], Name: display_name, dtype: object) :  2453
Series([], Name: display_name, dtype: object) :  2252
Series([], Name: display_name, dtype: object) :  868
Series([], Name: display_name, dtype: object), Series([], Name: display_name, dtype: object) :  653
Series([], Name: display_name, dtype: object), Series([], Name: display_name, dtype: object) :  38
Series([], Name: display_name, dtype: object), Series([], Name: display_name, dtype: object) :  36
Series([], Name: display_name, dtype: object), Series([], Name: display_name, dtype: object) :  26
Series([],

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [89]:
print("sum occurences: " + str(sum_occur))
print("ind occ: ")
print(ind_occur)

sum occurences: 27028
ind occ: 
[2453, 2458, 868, 2463, 2465, 2475, 2525, 2486, 2477, 2508, 2252, 0, 0, 0, 0]


In [ ]:
## UP-/DOWN-SAMPLEN

## Alle code hierna is alleen om uit te rekenen wat de proporties zijn
## Het werkt als volgt:
## Van het totale aantal voorkomens van een label (bv 11.000), wordt het gewenste afgetrokken:
## bv 11.000-2.000 = 9.000
## Dit aantal examples moet geskipt worden, maar dan alleen als het label alleen voorkomt
## Daarom wordt de target proportie van het totale individuutjes (target_ind) als volgt berekend:
## Stel ind_occ (aantal keer alleen voorkomend) is 10.000, waarvan we er 9.000 niet willen hebben.
## (10.000-9.000)/10.000 = 0.1 is dan de kans dat een example wordt geschreven.

In [90]:
## Code die kolom toevoegt aan label-csv (proportion) en deze in de forloop vult

## Aantal gewenste samples per label: 2500
target = 2500

## Aantal voorkomens en aantal voorkomens alleen
label_proportions["total"] = 0
label_proportions["ind_total"] = 0
label_proportions["proportion"] = 0
label_proportions["target_ind"] = 0

for i in range(mid_to_label_new.shape[0]):
    label_proportions.loc[i,"total"] = cnt_labels[i]
    label_proportions.loc[i,"ind_total"] = ind_occur[i]   
    label_proportions.loc[i,"proportion"] = cnt_labels[i]/sum_occur
    label_proportions.loc[i,"target_ind"] = (ind_occur[i]-(cnt_labels[i]-target))/ind_occur[i]

ZeroDivisionError: division by zero

In [ ]:
## Schrijf bestandje weg
label_proportions.to_csv(name_label_prop_file, sep = ";")

In [ ]:
label_proportions

In [10]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## 1 tf-record heeft meerdere examples
## Deze functie checkt voor elk example of het een label heeft uit onze labelset

## Input: filename van tf-record (pad) en prefix (vanwege dubbele filenames)
## Result: schrijven van nieuw tf-record

## Geef prefix mee (om dubbele namen te voorkomen)
def readTfRecord(tfrecords_filename):
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        print(example)
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
       
        #if(labels[0]!=11):
        #    print(labels)
        break;

In [11]:
file = 'tfrecords/train_jungle_new/gun_train.tfrecord'

In [12]:
readTfRecord(file)

context {
  feature {
    key: "labels"
    value {
      int64_list {
        value: 11
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "-1LrH01Ei1w"
      }
    }
  }
}
feature_lists {
  feature_list {
    key: "audio_embedding"
    value {
      feature {
        bytes_list {
          value: "\226\014\246Q\350Odm\251\300V_\300\225\0004\305Q\177\304\023\377Vni\221hF\237\216\323tq9V\257\'M\257\005<\377\017\'H\\\003R\222\242@\033`\237C}\210\0006\335\264F\000\200_SZ\"\220\222-\355\377\370\273\017\000\260\210\274\025\021\377\220\0000\302\352\257?\260\377\225\207ysM\031\301\377T\036Y~\243\235\312\300\324\306\377\377\377^\330\215,\211=\377\234\2029\000\353\000\000\377"
        }
      }
      feature {
        bytes_list {
          value: "\226\016\244J\342As]\210\331_d\277\244\021;\276x\205\263\036\356RX_\237Ra\224|\306pfC\023\372E1gWI\377\000\027BE\027_\221\301X\000s\241B\217\241\000>\372\235y\000\225mY\032\027\233QB\360\377\326\231;-\250